In [1]:
import torch
from sklearn.model_selection import train_test_split
import pandas as pd

from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

import timeit

import data_related as data
import bpe
import utils
import pretrained_model as pm
import data_loader as dl
import trainer
import initializer as init
import tester
import model_util as mu

In [2]:
first_np, second_np, label_np  = data.getData()

avg length / 2: 7


In [3]:
first_np, second_np = utils.process_splitted(first_np, second_np)

linearized complete!


In [4]:
first_ls, second_ls, ch2idx, max_len = utils.tokenize(first_np, second_np)
first2idx_np, second2idx_np = utils.encode(first_ls, second_ls, ch2idx, max_len)

done tokenizing both data!
saved ch2idx to file!
encoding comlete!


In [5]:
# pretrained_word2vec = pm.load_pretrained_model(ch2idx)
# pretrained_word2vec = torch.tensor(pretrained_word2vec)

In [6]:
train_first, test_first, train_second, test_second, train_labels, test_labels = train_test_split(
    first2idx_np, second2idx_np, label_np, test_size = 0.1, random_state = 43
)

train_first, val_first, train_second, val_second, train_labels, val_labels = train_test_split(
    train_first, train_second, train_labels, test_size = 0.1, random_state = 43
)

In [7]:
train_dataloader, val_dataloader, test_dataloader = dl.data_loader(
                                                                   train_first,
                                                                   train_second,
                                                                   val_first,
                                                                   val_second,
                                                                   test_first,
                                                                   test_second,
                                                                   train_labels,
                                                                   val_labels,
                                                                   test_labels,
                                                                   batch_size=100000)

In [8]:
# PyTorch TensorBoard support
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('../tensorboard/final/related')

In [9]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: NVIDIA GeForce RTX 3070


In [10]:
# ====================
# set parameters here
# ====================

title = 'related_test_1'
epochs = 40

input_size=len(ch2idx)
embed_dim = 300
hidden_size = 800
num_classes = 2
rnn_layers = 1

num_filters = [100, 200, 100]
kernel_sizes = [15, 21, 114]

dropout = 0.2

learning_rate = 0.001
weight_decay = 1e-4

model_name="RNN"
optim_name="Adam"
loss_fn_name="CEL"

pretrained_model=None
freeze_embedding=False,

In [11]:
trainer.set_seed(42)

model, optimizer, loss_fn = init.initialize_model(
    vocab_size=input_size,
    embed_dim=embed_dim,
    hidden_size=hidden_size,
    num_classes=num_classes,
    rnn_layers=rnn_layers,
    num_filters=num_filters,
    kernel_sizes=kernel_sizes,
    dropout=dropout,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    model_name=model_name,
    optim_name=optim_name,
    loss_fn_name=loss_fn_name,
    pretrained_model=pretrained_model,
    freeze_embedding=freeze_embedding,
    device=device,
)

print(model)

doing without pretrained model!!!


/home/arise/anaconda3/lib/python3.9/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


OCR_rnn(
  (emb): Embedding(117, 300)
  (lstm1): RNN(
    (rnn): LSTM(300, 800, batch_first=True, dropout=0.2, bidirectional=True)
  )
  (lstm2): RNN(
    (rnn): LSTM(300, 800, batch_first=True, dropout=0.2, bidirectional=True)
  )
  (fc1): Linear(in_features=3200, out_features=300, bias=True)
  (fc2): Linear(in_features=300, out_features=2, bias=True)
  (dp1): Dropout(p=0.2, inplace=False)
  (dp2): Dropout(p=0.2, inplace=False)
)


In [12]:
start_time = timeit.default_timer()

trainer.train(
    epochs=epochs,
    title=title,
    writer=writer,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    device=device,
    model=model,
    optimizer=optimizer,
    loss_fn=loss_fn
)

end_time = (timeit.default_timer() - start_time) / 60.0

Start training...

 Epoch  |  Train Loss  | Train Acc  | Val Loss | Val Acc | Elapsed
--------------------------------------------------------------------------------
   1    |   0.485213   | 76.024938  | 0.432376 | 79.47  | 581.29
   2    |   0.449654   | 78.547469  | 0.418240 | 80.39  | 584.89
   3    |   0.441215   | 79.148827  | 0.415006 | 80.74  | 549.65
   4    |   0.436649   | 79.472160  | 0.407218 | 81.19  | 532.06
   5    |   0.433995   | 79.634198  | 0.410090 | 80.86  | 535.36
   6    |   0.431434   | 79.780494  | 0.400440 | 81.76  | 567.74


In [ ]:
mu.saveModel(title, model)

In [ ]:
model = mu.getModel(title)
print(model)

In [ ]:
loss, acc = tester.test(test_dataloader=test_dataloader,
                        device=device,
                        model=model,
                        title=title)

In [ ]:
with open('../result/final_related', 'a') as f:
        text = title + '\t |\tloss: ' + str(loss) + '\t |\tacc: ' + str(acc) + '\t |\t time: ' + str(round(end_time, 3)) + ' min\n'
        f.write(text)

In [ ]:
mu.graphModel(train_dataloader, model, writer)

In [ ]:
max_len